In [85]:
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy.sparse as sparse
from scipy.linalg import eigh
import scipy.linalg
import matplotlib.pyplot as plt
from qutip import *
import math

In [86]:
#plt.rcParams['figure.figsize']=8.5,5.1
plt.rcParams['font.size']=20
plt.rcParams['lines.linewidth']=1.5
plt.rcParams['text.usetex']=True

In [87]:
#Construction of necessary elements of theta and phi Hilbert spaces

# For phi

N=50
a_phi=np.diag(np.sqrt(np.arange(1, N+1)), 1)
ad_phi=np.diag(np.sqrt(np.arange(1, N+1)), -1)
N_phi=ad_phi@a_phi
Id_N=np.eye(N+1)

# For theta

M=20
Id_M=np.eye(2*M+1)
n_theta=np.flipud(np.fliplr(np.diag(np.arange(-M,M+1), 0)))

#Values of equilibrium

phi_ext_val=np.pi
ng_val=0.5

#Qubit parameters 1 

E_Cs=1.0
E_Cj=1.0
E_L=1.0
E_J=1.0
alpha=(1j*(E_Cj/E_L)**(1.0/4.0))

# Qubit parameters 2

E_Cs_2=(1.0/20)
E_Cj_2=1.0
E_L_2=(1.0/16)
E_J_2=(1.0/5.0)
alpha2=(1j*(E_Cj_2/E_L_2)**(1.0/4.0))

#FOR EXPECTATION VALUES

#PHI
print("PHI")
print((E_Cj/E_L)**(1.0/4.0)*(a_phi+ad_phi))
phi=np.kron(Id_M,((E_Cj/E_L)**(1.0/4.0))*(a_phi+ad_phi))
print("kron PHI")
print(phi)
phi2=np.kron(Id_M,((E_Cj_2/E_L_2)**(1.0/4.0))*(a_phi+ad_phi))

#COS_THETA
costheta=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
costheta[0][2*M]+=1
costheta[2*M][0]+=1
print(costheta)
print("COS")
cos_theta=np.kron(costheta,Id_N)
print("kron COS")
print(cos_theta)

#N_PHI
N_phi_extended=np.kron(Id_M,N_phi)

#N_theta
n_theta_extended=np.kron(n_theta,Id_N)


#FOR SAVING VALUES OF CALCULATIONS

#1ST VALUES

ng1=np.arange(0.495,0.506,0.01)
print(ng1)
a=np.pi-(np.pi*0.01)
b=np.pi+(np.pi*0.01 + 0.001)
phi_ext1=np.arange(a,b,(0.01*np.pi)*2)
print(phi_ext1)
Energies=[]
Eigenvectors=[]

Expvals_costheta_NG=[]
Expvals_phi_NG=[]
Expvals_N_phi_NG=[]
Expvals_n_theta_NG=[]

Expvals_costheta_PHIEXT=[]
Expvals_phi_PHIEXT=[]
Expvals_N_phi_PHIEXT=[]
Expvals_n_theta_PHIEXT=[]

#2ND VALUES

ng2=np.arange(0.495,0.506,0.01)
phi_ext2=np.arange(a,b,(0.01*np.pi)*2)
Energies2=[]
Eigenvectors2=[]

Expvals2_costheta_NG=[]
Expvals2_phi_NG=[]
Expvals2_N_phi_NG=[]
Expvals2_n_theta_NG=[]

Expvals2_costheta_PHIEXT=[]
Expvals2_phi_PHIEXT=[]
Expvals2_N_phi_PHIEXT=[]
Expvals2_n_theta_PHIEXT=[]


PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
kron PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
[[0. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1

In [88]:
# Construction of the 0-pi Hamiltonian

def Hamiltonian_0pi(EJ,ECS,ECJ,EL,n,phi,alph):
    #Initial
    H=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Free Hamiltonian construction
    H1=4*ECS*((n_theta-n*Id_M)@(n_theta-n*Id_M))
    H_theta=np.kron(H1,Id_N)
    H2=2*np.sqrt(ECJ*EL)*(N_phi+0.5*Id_N)
    H_phi=np.kron(Id_M,H2)
    H_0=H_theta+H_phi
    #H_0=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Interaction Hamiltonian construction
    cos_theta_ext=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
    #cos_theta=np.diag(np.ones(2*M-1),2)+np.diag(np.ones(2*M-1),-2)
    cos_theta_ext[0][2*M]+=1
    cos_theta_ext[2*M][0]+=1
    #cos_phi=(0.5*((displace(N+1,alph)*(math.e**(-1j*phi/2)))+(displace(N+1,-alph)*(math.e**(1j*phi/2)))))*np.diag(np.ones(N+1),0)
    cos_phi=0.5*(scipy.linalg.expm(alph*(a_phi+ad_phi)-(Id_N*1j*phi/2))+scipy.linalg.expm(-alph*(a_phi+ad_phi)+(Id_N*1j*phi/2)))
    H_I=EJ*np.kron(cos_theta_ext,cos_phi.real)#SIN EL2 
    #H_I=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Total Hamiltonian
    H+=H_0-H_I
    return H

In [89]:
def calculate_expval(vect,operator):
    return np.conj(vect)@operator@vect

In [90]:
Vprueba=[1,1,0]
print(Vprueba)
matrix=[[0,1,0],[1,0,1],[0,1,0]]
print(matrix)
a=calculate_expval(Vprueba,matrix)
print(a)


[1, 1, 0]
[[0, 1, 0], [1, 0, 1], [0, 1, 0]]
2


## $\text{1. Plot for } E_{Cs} = E_{Cj} = E_L = 1$, $\varphi_{ext}= \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = i$.

In [91]:
# Calculations 1

for n1 in ng1:
    print("Step for ng = ",n1)
    H1=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,n1,phi_ext_val,alpha)
    print("Hamiltonian built, len(H)=",len(H1)**2)
    #vals,vects=np.linalg.eigh(H)
    valsNG, vectsNG = eigsh(H1, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsNG[0]," and ",valsNG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsNG[:,0]," and ",vectsNG[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsNG[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsNG[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsNG[:,0],phi)
    Expval_phi_1=calculate_expval(vectsNG[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsNG[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsNG[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsNG[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsNG[:,1],n_theta_extended)
    #TOTALS
    Expvals_costheta_NG.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_NG.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_NG.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_NG.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for ng =  0.495
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7550311282877298  and  1.7889259142999712
The eigenvectors of the GS and the 1st excited state are  [-4.14498257e-20 -3.41551326e-18 -8.33063089e-19 ...  1.89275468e-17
 -4.28507952e-17  1.22153588e-16]  and  [-2.44749585e-18 -3.69522884e-18 -3.67861915e-18 ... -1.96009930e-18
  9.93006070e-18  5.92851602e-19]
Eigenvalues and eigenvectors calculated
Step for ng =  0.505
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7550311282880668  and  1.7889259142998575
The eigenvectors of the GS and the 1st excited state are  [ 1.19913741e-18  1.64918824e-17  4.66654654e-18 ... -1.17712453e-17
  4.19933908e-18  1.89224004e-17]  and  [-7.76414430e-18 -2.35130545e-17 -5.05318465e-18 ...  2.31947609e-17
 -9.44627477e-18  1.91357296e-16]
Eigenvalues and eigenvectors calculated


In [92]:
# Calculations 1

for phi1 in phi_ext1:
    print("Step for phi_ext = ",phi1)
    H2=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,ng_val,phi1,alpha)
    print("Hamiltonian built, len(H)=",len(H2)**2)
    #vals,vects=np.linalg.eigh(H)
    valsPHI, vectsPHI = eigsh(H2, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsPHI[0]," and ",valsPHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsPHI[:,0]," and ",vectsPHI[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsPHI[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsPHI[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsPHI[:,0],phi)
    Expval_phi_1=calculate_expval(vectsPHI[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsPHI[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsPHI[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsPHI[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsPHI[:,1],n_theta_extended)
    #TOTALS
    Expvals_costheta_PHIEXT.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_PHIEXT.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_PHIEXT.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_PHIEXT.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for phi_ext =  3.1101767270538954
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.763178252558451  and  1.780734611521542
The eigenvectors of the GS and the 1st excited state are  [-1.08844621e-19  1.54787136e-18 -1.64679942e-19 ... -1.63684562e-17
  4.64168838e-17  7.57031329e-19]  and  [ 7.05707904e-20 -9.48357010e-19  1.12858916e-19 ... -2.73480639e-17
  2.77321456e-17  8.23536953e-18]
Eigenvalues and eigenvectors calculated
Step for phi_ext =  3.1730085801256913
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7631782525583561  and  1.7807346115216844
The eigenvectors of the GS and the 1st excited state are  [ 3.59954053e-18 -3.43293928e-18  2.35858138e-18 ... -1.07753388e-17
  9.81883868e-17 -6.06551783e-17]  and  [-5.04719947e-18 -4.61087695e-18  4.81704514e-19 ... -5.81709638e-22
  2.83826992e-17 -3.93922305e-16]
Eigenvalues and eigenvectors calculated


EXPECTATION VALUES FOR THE FIRST SET OF VALUES FOR THE ENERGIES

In [93]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals_costheta_NG)
print("Exp values of phi varying ng are ",Expvals_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [-3.022762294187597e-12, 2.995261890430263e-12, 2.1011030435333068e-14, -4.516097020057019e-14]
Exp values of phi varying ng are  [-1.5244051813152849e-12, 1.5656016035503275e-12, 4.903616971207764e-14, -1.5507449233682572e-13]
Exp values of N_phi varying ng are  [0.08693140982192102, 0.09220055517257782, 0.08693140982189938, 0.09220055517264435]
Exp values of n_theta varying ng are  [0.07369822649554705, 0.9209886856432679, 0.9263017735044603, 0.07901131435678296]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.073889114310878, -1.0705380216291664, -1.073889114310878, 1.070538021629191]
Exp values of phi varying phi_ext are  [0.5556498947237778, -0.5619624882127534, -0.5556498947238153, 0.5619624882128837]
Exp values of N_phi varying phi_ext are  [0.08873497723141124, 0.09026030865726747, 0.08873497723142484, 0.09026030865730672]
Exp values of n_theta varying phi_ext are  [0.5000000000016271, 0.4999999999983597, 0.4999

## $\text{2. Plot for } E_{Cs} = \frac{1}{20}$, $E_{CJ} = 1$, $E_L = \frac{1}{16}$, $\varphi_{ext} = \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = 2i$.

In [94]:
# Calculations 2

for n2 in ng2:
    print("Step for ng = ",n2)
    H3=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,n2,phi_ext_val,alpha2)
    print("Hamiltonian built, len(H)=",len(H3)**2)
    #vals2,vects2=np.linalg.eigh(H2)
    vals2NG, vects2NG = eigsh(H3, k=2, which='SA')
    #vals2,_=np.linalg.eigh(H3)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2NG[0]," and ",vals2NG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2NG[:,0]," and ",vects2NG[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2NG[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2NG[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2NG[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2NG[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2NG[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2NG[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2NG[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2NG[:,1],n_theta_extended)
    #TOTALS
    Expvals2_costheta_NG.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_NG.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_NG.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_NG.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for ng =  0.495
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2741644560315928  and  0.2761153906221432
The eigenvectors of the GS and the 1st excited state are  [-6.92902441e-18  1.88326070e-17 -1.85374012e-17 ...  3.68686650e-18
  3.69014036e-17 -8.55920167e-17]  and  [ 3.43614051e-18 -1.32368210e-17  1.38621949e-17 ... -3.11773268e-17
 -1.77707548e-17 -1.09689820e-16]
Eigenvalues and eigenvectors calculated
Step for ng =  0.505
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.27416445603159395  and  0.27611539062214396
The eigenvectors of the GS and the 1st excited state are  [-1.06700893e-17  4.75085097e-18 -1.73262499e-18 ...  3.33823202e-18
 -1.23727193e-16  5.05092928e-17]  and  [ 7.20227417e-18 -2.86979707e-19  1.95979571e-18 ...  1.10789004e-16
  8.95549602e-17  6.33677297e-17]
Eigenvalues and eigenvectors calculated


In [95]:
# Calculations 2

for phi2 in phi_ext2:
    print("Step for phi_ext = ",phi2)
    H4=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi2,alpha2)
    print("Hamiltonian built, len(H)=",len(H4)**2)
    #vals,vects=np.linalg.eigh(H)
    vals2PHI, vects2PHI = eigsh(H4, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2PHI[0]," and ",vals2PHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2PHI[:,0]," and ",vects2PHI[:,1])
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2PHI[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2PHI[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2PHI[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2PHI[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2PHI[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2PHI[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2PHI[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2PHI[:,1],n_theta_extended)
    #TOTALS
    Expvals2_costheta_PHIEXT.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_PHIEXT.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_PHIEXT.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_PHIEXT.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for phi_ext =  3.1101767270538954
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2747073723247168  and  0.2755640770043978
The eigenvectors of the GS and the 1st excited state are  [-9.99319226e-19  2.34669711e-18 -1.66936141e-18 ... -2.83304521e-17
 -4.30085209e-17 -4.99381717e-17]  and  [ 3.02005905e-18 -1.01765683e-18  1.77513558e-18 ...  1.62860191e-16
  3.10005902e-17  1.11494158e-16]
Eigenvalues and eigenvectors calculated
Step for phi_ext =  3.1730085801256913
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2747073723247447  and  0.2755640770044036
The eigenvectors of the GS and the 1st excited state are  [ 5.20030405e-19 -2.48708295e-18  4.54178432e-19 ...  6.50080089e-17
 -1.60474585e-17 -1.18277229e-16]  and  [-2.36815807e-18  2.69425347e-18 -6.72976193e-19 ...  8.54622653e-17
 -3.60937410e-17 -5.52662732e-17]
Eigenvalues and eigenvectors calculated


EXPECTATION VALUES FOR THE SECOND SET OF VALUES FOR THE ENERGIES

In [96]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals2_costheta_NG)
print("Exp values of phi varying ng are ",Expvals2_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals2_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals2_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals2_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals2_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals2_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals2_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [5.5550969235639966e-12, -5.566457381909192e-12, -1.8153545459288534e-11, 1.8161866588705553e-11]
Exp values of phi varying ng are  [1.136731977160889e-12, -1.1263252923123557e-12, -3.679534476868094e-12, 3.669889311033292e-12]
Exp values of N_phi varying ng are  [0.04402386741254535, 0.04424840052522372, 0.044023867412543514, 0.04424840052522558]
Exp values of n_theta varying ng are  [0.012071718350906722, 0.9875368300487353, 0.9879282816490964, 0.012463169951258039]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0837543531326792, -1.0800974073986538, -1.0837543531326774, 1.080097407398644]
Exp values of phi varying phi_ext are  [0.21752466351104865, -0.2187583312805243, -0.21752466351105046, 0.21875833128055494]
Exp values of N_phi varying phi_ext are  [0.04409616925676729, 0.04417101278913825, 0.04409616925676781, 0.0441710127891413]
Exp values of n_theta varying phi_ext are  [0.4999999999819598, 0.5000000000180314,